In [1]:
!pip install rasterio
!pip install pandas
!pip install geopandas
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install shapely
!pip install -q kaggle

In [2]:
import os
import glob
import gc
import random

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
from rasterio.transform import Affine

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union, transform

import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

Download dataset from Kaggle

In [3]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !kaggle datasets download -d isaienkov/deforestation-in-ukraine

# !mkdir dataset
# !unzip deforestation-in-ukraine.zip -d dataset

In [4]:
# Set the path to your dataset folder
dataset_folder = '/content/dataset'

# Use glob with recursive search to find all .SAFE directories
safe_dirs = glob.glob(os.path.join(dataset_folder, '**', '*.SAFE'), recursive=True)

# Ensure that only directories are included (in case there are files ending with .SAFE)
safe_dirs = [d for d in safe_dirs if os.path.isdir(d)]

In [5]:
safe_dirs[0:5]

['/content/dataset/S2A_MSIL1C_20190427T083601_N0207_R064_T36UXA_20190427T100851/S2A_MSIL1C_20190427T083601_N0207_R064_T36UXA_20190427T100851.SAFE',
 '/content/dataset/S2B_MSIL1C_20180815T084009_N0206_R064_T36UXA_20180815T123155/S2B_MSIL1C_20180815T084009_N0206_R064_T36UXA_20180815T123155.SAFE',
 '/content/dataset/S2A_MSIL1C_20180830T083601_N0206_R064_T36UXA_20180830T105527/S2A_MSIL1C_20180830T083601_N0206_R064_T36UXA_20180830T105527.SAFE',
 '/content/dataset/S2B_MSIL1C_20180904T083549_N0206_R064_T36UXA_20180904T123955/S2B_MSIL1C_20180904T083549_N0206_R064_T36UXA_20180904T123955.SAFE',
 '/content/dataset/S2A_MSIL1C_20180810T083601_N0206_R064_T36UXA_20180810T124435/S2A_MSIL1C_20180810T083601_N0206_R064_T36UXA_20180810T124435.SAFE']

Images with different weather conditions

In [ ]:
def show_raster(path):
    with rasterio.open(path, "r", driver="JP2OpenJPEG") as src:
        raster_img = src.read()
        raster_meta = src.meta

    raster_img = reshape_as_image(raster_img)
    plt.figure(figsize=(15,15))
    plt.imshow(raster_img)

A litle bit of clouds, No snow

In [ ]:
path_1 = "/content/dataset/S2A_MSIL1C_20160621T084012_N0204_R064_T36UYA_20160621T084513/S2A_MSIL1C_20160621T084012_N0204_R064_T36UYA_20160621T084513.SAFE/GRANULE/L1C_T36UYA_A005209_20160621T084513/IMG_DATA/T36UYA_20160621T084012_TCI.jp2"

show_raster(path_1)

A litle bit of clouds, Snow

In [7]:
path_2 = "/content/dataset/S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510/S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510.SAFE/GRANULE/L1C_T36UYA_A003350_20160212T084510/IMG_DATA/T36UYA_20160212T084052_TCI.jp2"

show_raster(path_2)

A lot of clouds, No snow

In [8]:
path_3 = "/content/dataset/S2A_MSIL1C_20190616T083601_N0207_R064_T36UYA_20190616T105922/S2A_MSIL1C_20190616T083601_N0207_R064_T36UYA_20190616T105922.SAFE/GRANULE/L1C_T36UYA_A020796_20190616T084207/IMG_DATA/T36UYA_20190616T083601_TCI.jp2"

show_raster(path_2)

In [15]:
# Function to resample the band if needed (assuming it exists)
def resample_band(src, shape):
    return src.read(out_shape=shape, resampling=rasterio.enums.Resampling.bilinear)

# # Resample TCI and NDVI to a lower resolution
# def resample_image(image, scale=0.5):
#     new_shape = (int(image.shape[0] * scale), int(image.shape[1] * scale))
#     return cv2.resize(image, new_shape, interpolation=cv2.INTER_LINEAR)

# Convert images from .jp2 to png
def save_image_as_png(image, band_name, granule_name, output_dir):
    output_path = os.path.join(output_dir, band_name, f"{granule_name}_{band_name}.png")
    plt.imsave(output_path, image, cmap='RdYlGn' if band_name == 'NDVI' else None)  # NDVI gets a colormap
    print(f"Saved {band_name} PNG to {output_path}")

In [17]:
# Function to compute NDVI
def compute_ndvi(nir_band, red_band):
    # Convert bands to float32 to avoid division issues
    nir = nir_band.astype('float32')
    red = red_band.astype('float32')
    # Avoid division by zero
    ndvi = np.where(
        (nir + red) == 0.,
        0,
        (nir - red) / (nir + red)
    )
    return ndvi

In [18]:
# Delete an image_matching folder if needed
!rm -r /content/image_matching

In [19]:
# Create the image_matching folder with subdirectories for each band
output_dir = '/content/image_matching'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'TCI'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'NDVI'), exist_ok=True)

In [20]:
weather_1 = "/content/dataset/S2A_MSIL1C_20160621T084012_N0204_R064_T36UYA_20160621T084513/S2A_MSIL1C_20160621T084012_N0204_R064_T36UYA_20160621T084513.SAFE"
weather_2 = "/content/dataset/S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510/S2A_MSIL1C_20160212T084052_N0201_R064_T36UYA_20160212T084510.SAFE"
weather_3 = "/content/dataset/S2A_MSIL1C_20190616T083601_N0207_R064_T36UYA_20190616T105922/S2A_MSIL1C_20190616T083601_N0207_R064_T36UYA_20190616T105922.SAFE"

three_safe_dirs = [weather_1, weather_2, weather_3]

In [21]:
# Loop through each .SAFE directory
for safe_dir in three_safe_dirs:
    print(f"Processing {safe_dir}")

    # Path to the IMG_DATA folder
    img_data_dir = os.path.join(safe_dir, 'GRANULE')
    granule_dirs = glob.glob(os.path.join(img_data_dir, '*'))

    for granule_dir in granule_dirs:
        img_data_path = os.path.join(granule_dir, 'IMG_DATA')

        # Load the required bands
        band_files = {
            'TCI': glob.glob(os.path.join(img_data_path, '*_TCI.jp2')),
            'B04': glob.glob(os.path.join(img_data_path, '*_B04.jp2')),  # Red band
            'B08': glob.glob(os.path.join(img_data_path, '*_B08.jp2')),  # NIR band
        }

        # Ensure all bands are found
        if all(len(band_files[key]) > 0 for key in band_files):
            # Open bands
            with rasterio.open(band_files['TCI'][0]) as tci_src, \
                 rasterio.open(band_files['B04'][0]) as b04_src, \
                 rasterio.open(band_files['B08'][0]) as b08_src:

                # Read TCI (RGB image directly)
                tci_image = tci_src.read()
                tci_image = reshape_as_image(tci_image)  # Reshape to RGB

                # Read B04 and B08 as single-channel images (grayscale)
                b04_image = b04_src.read(1)
                b08_image = b08_src.read(1)

                # Resample B04 and B08 to match TCI dimensions
                dst_width = tci_src.width
                dst_height = tci_src.height
                b04_resampled = resample_band(b04_src, (b04_src.count, dst_height, dst_width))[0]
                b08_resampled = resample_band(b08_src, (b08_src.count, dst_height, dst_width))[0]

                # Compute NDVI
                ndvi_image = compute_ndvi(b08_resampled, b04_resampled)

                # Plot all three bands in one row
                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                plt.imshow(tci_image)
                plt.title('TCI (True Color Image)')

                plt.subplot(1, 2, 2)
                plt.imshow(ndvi_image)
                plt.title('NDVI (Normalized Difference Vegetation Index)')

                plt.suptitle(f'NDVI for {os.path.basename(granule_dir)}')
                plt.show()

                # Save to the appropriate folder
                granule_name = os.path.basename(granule_dir)
                save_image_as_png(tci_image, 'TCI', granule_name, output_dir)
                save_image_as_png(ndvi_image, 'NDVI', granule_name, output_dir)

                # Free memory after each image is processed
                del tci_image, b04_resampled, b08_resampled, ndvi_image
                gc.collect()

        else:
            print(f"Required bands not found in {img_data_path}")

print("Processing completed.")

Output hidden; open in https://colab.research.google.com to view.

Send the folder to a google colab

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import shutil

# Define the source folder in Colab (e.g., the folder you want to copy)
source_folder = '/content/image_matching'

# Define the destination folder in Google Drive
destination_folder = '/content/drive/MyDrive/image_matching'

# Copy the entire folder to Google Drive
shutil.copytree(source_folder, destination_folder)

print(f"Folder {source_folder} copied to {destination_folder} successfully.")

Folder /content/image_matching copied to /content/drive/MyDrive/image_matching successfully.
